In [ ]:
import pandas as pd

train_data_with_cone = pd.read_parquet('../ml/processed_events_50k/raw_cone_data.parquet')

In [ ]:
import sys
sys.path.insert(0, "../")
from ml.generate_cone_data import normalize_cone_features
train_data_with_cone = normalize_cone_features(train_data_with_cone)

In [ ]:
# enable autoreloading of modules
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from src.plotting import plot_2d_scatter, plot_histogram, plot_records_per_plane, plot_cone_with_scatter, plot_y_values_per_plane, generate_3d_cone_animation
from src.data_exploration import calculate_max_records_events, select_event_data, print_y_values_per_plane
from src.featurization import calcualte_cone_parameters


# Prepare training data

Select input and output columns

In [ ]:
import sklearn as sk
from sklearn.model_selection import train_test_split

input_columns = [
            "primary_kinetic_energy",
            "sin_azimuth", "cos_azimuth", "sin_zenith", "cos_zenith"
        ]
output_columns = [
    'X_mean_min', 'Y_mean_min', 'Z_mean_min',
    'X_mean_max', 'Y_mean_max', 'Z_mean_max',
    'radius'
]
batch_identifier = "event_id"

Calculate meand and std of secondary features

In [ ]:
train_data_with_cone

In [ ]:
train_data_with_cone

In [ ]:
train_data_with_cone.shape

Impute nan values

In [ ]:
# validate na values before imputation
print(f"NA values before imputation:\n{train_data_with_cone.isnull().sum()}")

# impute na with 0
train_data_with_cone = train_data_with_cone.dropna()

# validate na values after imputation
print(f"NA values after imputation:\n{train_data_with_cone.isnull().sum()}")

In [ ]:
print(f"Event stats columns: {train_data_with_cone.columns}")
print(f"Input columns: {input_columns}")
print(f"Output columns: {output_columns}")

# Define NN using PINA

Test train split

In [ ]:
from sklearn.preprocessing import StandardScaler

# split inputs and outputs 
X = train_data_with_cone[input_columns]
y = train_data_with_cone[output_columns]


# y_normalized = y - y.mean()
# y_max = y_normalized.max().max()
# y_normalized = y_normalized / y_max

# create scaler for each output column
y_scaler = StandardScaler()

# fit and transform each column separately
y_normalized = y_scaler.fit_transform(y)

PINA

In [ ]:
# import seaborn as sns

# temp_input = pd.concat([y_normalized, X])
# # columns to exclude from the pairplot
# exclude_cols = ['event_id', 'primary_kinetic_energy', 'min_plane', 'max_plane']

# # build dataframe for plotting (only numeric columns)
# plot_df = temp_input.drop(columns=[c for c in exclude_cols if c in temp_input.columns])
# plot_df = plot_df.select_dtypes(include='number')

# sns.pairplot(plot_df)

In [ ]:
from pina import Trainer, Condition, LabelTensor
from pina.problem import AbstractProblem
from src.nn import Model
from pina.optim import TorchOptimizer, TorchScheduler
from pina.solver import DeepEnsembleSupervisedSolver
from pina.callback import MetricTracker
import torch



In [ ]:
x_pina = LabelTensor(X.values, input_columns)
y_pina = LabelTensor(y_normalized, output_columns)

In [ ]:
class BayesianProblem(AbstractProblem):

    output_variables = output_columns
    input_variables = input_columns
    conditions = {"data": Condition(input=x_pina, target=y_pina)}


problem = BayesianProblem()

In [ ]:
n_models = 2

# define problem & data (step 1)


models = [
  Model(
      input_dimensions=len(problem.input_variables),
      output_dimensions=len(problem.output_variables),
      layers=[100, 100, 100],
      func=torch.nn.Tanh
  )
  for _ in range(n_models)
]

optimizers = [
     TorchOptimizer(torch.optim.RAdam, lr=0.005)
     for _ in range(n_models)
]

schedulers = [
    TorchScheduler(torch.optim.lr_scheduler.MultiStepLR, milestones=[50, 100, 300, 500, 800], gamma=0.5)
    for _ in range(n_models)
]

In [ ]:
solver = DeepEnsembleSupervisedSolver(
    problem,
    models,
    optimizers=optimizers,
    schedulers=schedulers
    )

In [ ]:
# create the trainer
trainer = Trainer(
    solver=solver,  # The ensemble solver
    max_epochs=1500,  # Maximum number of training epochs
    logger=True,  # Enables logging (default logger is CSVLogger)
    callbacks=[MetricTracker()],  # Tracks training metrics using MetricTracker
    accelerator="cpu",  # Use CPU for training, alternative is "gpu" for GPU training
    train_size=0.7,  # Fraction of the dataset used for training (70%)
    test_size=0.2,  # Fraction of the dataset used for testing (20%)
    val_size=0.1,  # Fraction of the dataset used for validation (10%)
)

import time

start = time.time()
# train
trainer.train()
end = time.time()
print(f"Training time: {end - start} seconds")

Plot loss

In [ ]:
# inspecting final loss
trainer.logged_metrics

In [ ]:
# plot loss
trainer_metrics = trainer.callbacks[0].metrics
loss = trainer_metrics["train_loss"]
epochs = range(len(loss))
plt.plot(epochs, loss.cpu())
# plotting
plt.xlabel("epoch")
plt.ylabel("loss")
plt.yscale("log")

# Test NN: Plot errors in output/target features

In [ ]:
all_outputs = None
all_targets = None

trainer.data_module.setup("test")
with torch.no_grad():
    for data in trainer.data_module.test_dataloader():
    # for data in trainer.data_module.train_dataloader():
        inputs, target = data[0][1]["input"], data[0][1]["target"]
        outputs = solver(inputs)
        
        if all_outputs is None:
            all_outputs = LabelTensor(outputs, labels=output_columns)
            all_targets = target
        else:
            all_outputs.append(LabelTensor(outputs, labels=output_columns))
            all_targets.append(target)
        break

In [ ]:
# plot targets vs predictions for validation set
y_mean, y_std = all_outputs.mean(0).detach(), all_outputs.std(0).detach()
true_output = all_targets.detach()

plt.figure(figsize=(18, 10))
# use 3 columns per row
for i, col in enumerate(output_columns):
    plt.subplot(len(output_columns)//4+1, 4, i+1)
    plt.scatter(true_output[:, i], y_mean[:, i], alpha=0.5)#, s=20*y_std[:, i]/y_std.max())
    plt.plot([true_output[:, i].min(), true_output[:, i].max()],
             [true_output[:, i].min(), true_output[:, i].max()], 'r--')
    
    plt.xlim([-1,1])
    plt.ylim([-1,1])
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title(col)
plt.tight_layout(pad=1.0)
